In [127]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib as ta
from scipy.signal import argrelextrema
from scipy.signal import find_peaks
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")

if not mt5.initialize(path = r'C:\Program Files\Zero Markets MetaTrader 5 Terminal\terminal64.exe'):
    print("initialize() failed")
    mt5.shutdown()

In [433]:
ativo = 'AUDUSD'
df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 30000))
df['Data'] = pd.to_datetime(df['time'], unit='s').apply(lambda x: str(x))
df = df[['Data','open','high','low','close','real_volume']]
df.columns = ['Data','Open','High','Low','Close','Vol']
df = df.rename(columns = {'real_volume':'vol'})
df['RSI'] = ta.RSI(df['Close'],timeperiod = 14)
df = df.iloc[:10000]

In [434]:
time_series = df['Close']
indices = find_peaks(time_series,distance = 15)[0]

fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series,
    mode='lines',
    name='Original Plot'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))

fig.show()

In [443]:
time_series = df['Close']
indices = find_peaks(time_series,distance = 10)[0]
indices_2 = find_peaks(time_series,distance = 1)[0]

final = []
valor_pra_frente = 30
for ind in indices:

    for n_ind in range(ind+1+10,ind+valor_pra_frente):
        try:
            if df.loc[(n_ind,'Close')] >= df.loc[(ind,'Close')] and n_ind in indices_2 and df.loc[(n_ind+2,'Close')] <= df.loc[(ind,'Close')]  and  df.loc[(n_ind,'RSI')] > 60:
                #and df.loc[(n_ind,'RSI')] < df.loc[(ind,'RSI')]
                final.extend([ind,n_ind])
                break
        except:
            break


In [444]:
indices = final
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series,
    mode='lines',
    name='Original Plot'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks'
))

fig.show()

In [445]:
lista_signal = [0] * len(df)
lista_stop = [0] * len(df)
lista_tp = [0] * len(df)
lista_close = df['Close'].values


for i in final:

    try:
        valor_pra_frente = 1
        tam_stop = 0.0002
        tamanho_stop = lista_close[i] - lista_close[i+valor_pra_frente]
        if tamanho_stop > 0.0003 and tamanho_stop < 0.0004:
            lista_signal[i+valor_pra_frente] = 2
            lista_stop[i+valor_pra_frente] = lista_close[i]
            #lista_stop[i+valor_pra_frente] = lista_close[i] + tamanho_stop
            lista_tp[i+valor_pra_frente] = lista_close[i+valor_pra_frente] - (tamanho_stop)
    except:
        pass

df['signal'] = lista_signal
df['stop'] = lista_stop
df['tp'] = lista_tp
df['tamanho'] = df['stop'] - df['Close']

'''df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index('Data')'''

"df['Data'] = pd.to_datetime(df['Data'])\ndf = df.set_index('Data')"

In [446]:
from backtesting import Strategy, Backtest
def SIGNAL():
    return df.signal

class MyCandlesStrat(Strategy):
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #self.ema = self.I(ta.EMA, self.data.Close, 89)
        #self.rsi = self.I(ta.RSI, self.data.Close, 14)
        #self.teste = self.I(teste,self.data.Close)

    def next(self):
        super().next() 
        if self.signal1==1:
            sl1 = self.data.stop[-1]
            tp1 = self.data.tp[-1]
            self.buy(sl=sl1, tp=tp1)
        elif self.signal1==2:
            sl1 = self.data.stop[-1]
            tp1 = self.data.tp[-1]
            self.sell(sl=sl1, tp=tp1)

bt = Backtest(df, MyCandlesStrat, cash=100000000, commission=0,hedging=True)
stats = bt.run()


bt.plot()

Row(id='75839', ...)